## imports

In [1]:
import re
import json

alphanumeric='abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789'
dot =  '('+('|'.join(alphanumeric))+')'

# print(dot)


# Regex to Postfix

In [2]:
def validate_regex(regex):
    try:
       re.compile(regex)
       return True
    except:
       return False 
    



def handle_dot(regex):
   return re.sub(r'\.', dot, regex)

def handle_range(regex):
    new_regex = ""
    replace = False
    start = None
    end= None
    for i in range(len(regex)):
        if regex[i] == '[':
            new_regex += '('
            replace =True
        elif regex[i]== ']':
             new_regex += ')'
             replace =False
        elif regex[i] == '-':
            replace = True
            start = alphanumeric.index(regex[i-1])
            end = alphanumeric.index(regex[i+1])
            for j in range(start+1,end):
                new_regex += alphanumeric[j]
                if(replace and regex[i+1] != ']'):
                    new_regex+='|'
        else:
           new_regex += regex[i] 
           if(replace and regex[i+1] != ']'):
                    new_regex+='|'


    return new_regex  


def concatenate(regex):
    new_regex = ""
    for i in range(len(regex)-1):
        new_regex += regex[i] 
        if regex[i] in ['*','+','?',')',']'] and regex[i+1] not in ['*','+','?',')',']','.', '|']:
            new_regex+="."
        elif regex[i] in alphanumeric and  (regex[i+1] in alphanumeric or regex[i+1] in ['(','[']):
            new_regex+="."
    
    new_regex += regex[-1]

    return new_regex

def infix_to_postfix(regex):
    queue = []
    stack = []
    operators={'(':0,"|":1, ".":2, "?":3, "+":4, "*":5}

    for i in range(len(regex)):
        if regex[i] == '(':
            stack.append(regex[i])
        elif regex[i] == ')':
            while stack[-1] != "(":  
                queue.append(stack.pop()) 
            stack.pop()     
        elif regex[i] in operators:
            while stack  and operators[regex[i]] <= operators[stack[-1]]:
                queue.append(stack.pop())
            stack.append(regex[i])

        else :
            queue.append(regex[i])    
    
    while stack:
        queue.append(stack.pop())

    return ''.join(queue  )  


def to_postfix(regex):
    if(validate_regex(regex)):
       regex = handle_dot(regex)
       print(regex)
       regex = handle_range(regex)
       print(regex)
       regex = concatenate(regex)
       print(regex)
       regex = infix_to_postfix(regex)
       print(regex)

       return regex


    else:
        print('sorry you have entered invalid regex! ')    


    

# print(handle_dot('ab.c'))
# print(handle_range("[a-z]+abc[Abc]*[1-5]"))
# print(concatenate("a+b*|c(5.f)"))
# print(infix_to_postfix("(A+.B*)?.(C|D)"))
print(to_postfix("."))        


(a|b|c|d|e|f|g|h|i|j|k|l|m|n|o|p|q|r|s|t|u|v|w|x|y|z|A|B|C|D|E|F|G|H|I|J|K|L|M|N|O|P|Q|R|S|T|U|V|W|X|Y|Z|0|1|2|3|4|5|6|7|8|9)
(a|b|c|d|e|f|g|h|i|j|k|l|m|n|o|p|q|r|s|t|u|v|w|x|y|z|A|B|C|D|E|F|G|H|I|J|K|L|M|N|O|P|Q|R|S|T|U|V|W|X|Y|Z|0|1|2|3|4|5|6|7|8|9)
(a|b|c|d|e|f|g|h|i|j|k|l|m|n|o|p|q|r|s|t|u|v|w|x|y|z|A|B|C|D|E|F|G|H|I|J|K|L|M|N|O|P|Q|R|S|T|U|V|W|X|Y|Z|0|1|2|3|4|5|6|7|8|9)
ab|c|d|e|f|g|h|i|j|k|l|m|n|o|p|q|r|s|t|u|v|w|x|y|z|A|B|C|D|E|F|G|H|I|J|K|L|M|N|O|P|Q|R|S|T|U|V|W|X|Y|Z|0|1|2|3|4|5|6|7|8|9|
ab|c|d|e|f|g|h|i|j|k|l|m|n|o|p|q|r|s|t|u|v|w|x|y|z|A|B|C|D|E|F|G|H|I|J|K|L|M|N|O|P|Q|R|S|T|U|V|W|X|Y|Z|0|1|2|3|4|5|6|7|8|9|


# Postfix to NFA

In [59]:
class edge:
    def __init__(self):
       self.label=None
       self.destination=None


class state:
       def __init__(self):
        self.label=None
        self.outgoing_edges=[]




class NFA:    
       def __init__(self, start, accept,states):
        self.start_state = start
        self.acceptance_state = accept
        self.states = states
        

def construct_nfa (label, id, stack):
    start = state()
    acceptance = state()
    start.label = "S"+str(id)
    acceptance.label = "S"+str(id+1)

    Edge = edge()
    Edge.label = label
    Edge.destination = acceptance 
    start.outgoing_edges.append(Edge)

    result_nfa = NFA(start, acceptance, [start,acceptance])
    stack.append(result_nfa)
    #print(result_nfa.inner_states)

    return result_nfa, id+2


def nfa_concatination (stack):
   
    
    nfa2 = stack.pop()
    nfa1 = stack.pop()


    Edge = edge()
    Edge.label = 'ε'
    Edge.destination = nfa2.start_state
    nfa1.acceptance_state.outgoing_edges.append(Edge)

    #print(nfa1)
    #print(nfa1.inner_states)

    resultnfa = NFA(nfa1.start_state,  nfa2.acceptance_state, nfa1.states + nfa2.states)
    stack.append(resultnfa)
    return resultnfa


def nfa_or (stack, id):
  nfa1 = stack.pop()
  nfa2 = stack.pop()
  newStart = state()
  newStart.label= "S"+str(id)
  newEnd = state()
  newEnd.label = "S"+str(id+1)

  Edge1 = edge()
  Edge1.label = 'ε'
  Edge1.destination = nfa1.start_state

  Edge2 = edge()
  Edge2.label = 'ε'
  Edge2.destination = nfa2.start_state

  newStart.outgoing_edges.append(Edge1)
  newStart.outgoing_edges.append(Edge2)


  Edge3 = edge()
  Edge3.label = 'ε'
  Edge3.destination = newEnd

  nfa1.acceptance_state.outgoing_edges.append(Edge3)

  Edge4 = edge()
  Edge4.label = 'ε'
  Edge4.destination = newEnd
  nfa2.acceptance_state.outgoing_edges.append(Edge4)


  result = NFA(newStart, newEnd, [newStart,newEnd]+ nfa1.states + nfa2.states)
  stack.append(result)
  return result, id+2



def nfa_zero_or_more (stack, id):
  nfa1 = stack.pop()



  newStart = state()
  newStart.label = "S"+str(id)
  newEnd = state()
  newEnd.label = "S"+str(id+1)

  Edge1 = edge()
  Edge1.label = 'ε'
  Edge1.destination = newStart
  nfa1.acceptance_state.outgoing_edges.append(Edge1)



  Edge2 = edge()
  Edge2.label = 'ε'
  Edge2.destination=nfa1.start_state
  newStart.outgoing_edges.append(Edge2)

  Edge3 = edge()
  Edge3.label = 'ε'
  Edge3.destination = newEnd
  nfa1.acceptance_state.outgoing_edges.append(Edge3)



  Edge4 = edge()
  Edge4.label = 'ε'
  Edge4.destination = newEnd
  newStart.outgoing_edges.append(Edge4)
  
  result = NFA (newStart, newEnd, [newStart, newEnd] + nfa1.states)
  stack.append(result)
  return result, id+2


def nfa_one_or_more (stack, id):
   nfa1 = stack.pop()



   newStart = state()
   newStart.label = "S"+str(id)
   newEnd = state()
   newEnd.label = "S"+str(id+1)

   Edge1 = edge()
   Edge1.label ='ε'
   Edge1.destination = newStart
   nfa1.acceptance_state.outgoing_edges.append(Edge1)

   Edge2 = edge()
   Edge2.label='ε'
   Edge2.destination=nfa1.start_state
   newStart.outgoing_edges.append(Edge2)

   Edge3 = edge()
   Edge3.label='ε'
   Edge3.destination = newEnd
   nfa1.acceptance_state.outgoing_edges.append(Edge3)


   result = NFA (newStart, newEnd, [newStart, newEnd] + nfa1.states)
   stack.append(result)
   return result, id+2

def nfa_zero_or_one (stack, id):
  nfa1 = stack.pop()

  newStart = state()
  newStart.label = "S"+str(id)
  newEnd = state()
  newEnd.label = "S"+str(id+1)

  Edge1 = edge()
  Edge1.label='ε'
  Edge1.destination=nfa1.start_state
  newStart.outgoing_edges.append(Edge1)

  Edge2 = edge()
  Edge2.label='ε'
  Edge2.destination = newEnd
  nfa1.acceptance_state.outgoing_edges.append(Edge2)

  
  Edge2 = edge()
  Edge2.label='ε'
  Edge2.destination = newEnd
  newStart.outgoing_edges.append(Edge2)



  result = NFA (newStart, newEnd, [newStart, newEnd] + nfa1.states)
  stack.append(result)
  return result, id+2

def postfix_to_nfa(postfix):
     stack = []
     id = 1

     for i in range(len(postfix)):
         if postfix[i] in alphanumeric:
           _ ,id =  construct_nfa(postfix[i],id,stack)
         elif postfix[i] == '.':   
             nfa_concatination(stack) 
         elif postfix[i] == '|':   
            _,id = nfa_or(stack,id)   
         elif  postfix[i] == '*':  
              _,id = nfa_zero_or_more(stack,id)  
         elif  postfix[i] == '+':  
              _,id = nfa_one_or_more(stack,id)    
         elif  postfix[i] == '?':  
               _,id = nfa_zero_or_one(stack,id) 
         else:
              raise ValueError(f"an invialid operator is entered {postfix[i]}")   
              
     result = stack.pop()
     return result


def convert_nfa_to_dictionary(nefa:NFA):
  outputJson = dict()
  outputJson["startingState"] = nefa.start_state.label
  for stat in nefa.states:
    stateDict = dict()
    if stat == nefa.acceptance_state:
      stateDict["isTerminatingState"] = True
    else:
      stateDict["isTerminatingState"] = False
    for edg in stat.outgoing_edges:
      if(edg.label in stateDict.keys()):
        stateDict[edg.label] = [] + [stateDict[edg.label]] + [edg.destination.label]
      else:
        stateDict[edg.label] = edg.destination.label
    outputJson[stat.label] = stateDict
  return outputJson  

    










# render graph and json

In [57]:
from graphviz import Digraph

def save_json_file (dict,filename):
  
    json_object = json.dumps(dict, indent=6,ensure_ascii=False)
    with open(filename + ".json", "w", encoding="utf-8") as outfile:
        outfile.write(json_object)

def render_graph(finite_automata, filename, attr):
    '''
    Renders the NFA as a graph and saves it to a file.
    '''
    gra = Digraph(graph_attr={'rankdir':'LR', 'bgcolor': attr["bgcolor"]})
    # Add the nodes to the graph
    for state in finite_automata:
        if state == "startingState":
            gra.node(state, style='invisible')
        else:
            shape = "doublecircle" if finite_automata[state]["isTerminatingState"] else "circle"
            gra.node(state, shape=shape, style='filled', fillcolor= attr["node_fillcolor"])

    # Add the edges to the graph
    for from_state in finite_automata:
        if from_state == "startingState":
            gra.edge(tail_name=from_state, head_name=finite_automata["startingState"], color=attr["edge_color"])
            continue

        for input in finite_automata[from_state]:
            if input == "isTerminatingState":
                continue
            to_states = finite_automata[from_state][input]
            # Decide to Draw edge or edges based on 
            # whether have a single destination or a list of destinations
            if type(to_states) == list:
                for to_state in to_states:
                    gra.edge(tail_name=from_state, head_name=to_state, label=input, color=attr["edge_color"])
            else:
                gra.edge(tail_name=from_state, head_name=to_states, label=input, color=attr["edge_color"])

    #gra.attr(label="The NFA for the pattern: " + pattern, fontcolor=attr["label_fontcolor"], fontname='bold', fontsize=attr["label_fontsize"])
    gra.format= 'png'
    gra.render(filename, view=True)

graph_attr = { 
        "bgcolor": "#ffffff",
        "node_fillcolor": "#ffffff",
        "edge_color": "#000000",
        "label_fontcolor": "black",
        "label_fontsize": "20"
        }        

In [61]:
postfix = to_postfix("(a+b)")
nfa1 = postfix_to_nfa(postfix)
nfa_dict = convert_nfa_to_dictionary(nfa1)
save_json_file(nfa_dict,"nfa1")


(a+b)
(a+b)
(a+.b)
a+b.


# NFA to DFA

In [5]:
def get_epsilon_closure(nfa, input_state):

    stack = [input_state]
    closure = set()

    while stack:

        current_state = stack.pop()
        closure.add(current_state)

        if "ε" in nfa[current_state]:

            next_states = nfa[current_state]["ε"]
            if isinstance(next_states, str):
                next_states = [next_states]

            for state in next_states:
                if state not in closure:
                    stack.append(state)
                    closure.add(state)

    return tuple(sorted(closure))

def convert_nfa_to_dfa(nfa):

    nfa_start_state = nfa["startingState"]
    dfa_start_state = get_epsilon_closure(nfa,nfa_start_state)

    dfa = {}
    dfa["startingState"] = dfa_start_state

    stack = [dfa_start_state]

    while stack:
        current_state = stack.pop()
        dfa[current_state] = {}     #current_state = (s0,s1,s2) 
        dfa[current_state]["isTerminatingState"] = False

        for state in current_state:
            nfa_state = nfa[state]   # nfa_state -> dic of s0 in nfa

            for sub_key in nfa_state:

                if sub_key == "isTerminatingState":
                    dfa[current_state]["isTerminatingState"] |= nfa_state["isTerminatingState"]

                elif sub_key != "ε": # edges of s0
                    next_states = get_epsilon_closure(nfa,nfa_state[sub_key])
                    if sub_key in dfa[current_state]: # if edge already in dfa state
                        temp_set= set(dfa[current_state][sub_key])
                        temp_set.update(next_states)
                        dfa[current_state][sub_key] = tuple(sorted(temp_set))
                    else:
                        dfa[current_state][sub_key] = next_states
        
        for new_sub_key in dfa[current_state]:
            if new_sub_key != "isTerminatingState":
                new_added_state = dfa[current_state][new_sub_key]
                if new_added_state not in dfa:
                    stack.append(new_added_state)

    return dfa

def get_final_dfa(dfa):

    # create a dict for states with their new names
    new_states_names = {}
    for i,state in enumerate(dfa):
        if state != "startingState":
            new_states_names[state] = "S"+str(i-1)
    print(new_states_names)

    # replace states with their new names
    new_dfa = dfa.copy()

    for key in dfa:
        if key != "startingState":

            for sub_key in dfa[key]:
                if sub_key !="isTerminatingState":
                    old_state = dfa[key][sub_key]  #('S1', 'S3')
                    new_state = new_states_names[old_state] #'S0'
                    new_dfa[key][sub_key] = new_state

            old_state = key  #('S1', 'S3')
            new_state = new_states_names[old_state] #'S0'
            new_dfa[new_state]= new_dfa[old_state]
            del new_dfa[key]

        else:
            old_state = dfa[key]
            new_state = new_states_names[old_state]
            new_dfa[key] = new_state

    return new_dfa

In [6]:
test_nfa= {
      "startingState": "S61",
      "S61": {
            "isTerminatingState": False,
            "ε": [
                  "S59",
                  "S57"
            ]
      },
      "S62": {
            "isTerminatingState": False,
            "ε": "S63"
      },
      "S59": {
            "isTerminatingState": False,
            "9": "S60"
      },
      "S60": {
            "isTerminatingState": False,
            "ε": "S62"
      },
      "S57": {
            "isTerminatingState": False,
            "ε": [
                  "S55",
                  "S53"
            ]
      },
      "S58": {
            "isTerminatingState": False,
            "ε": "S62"
      },
      "S55": {
            "isTerminatingState": False,
            "8": "S56"
      },
      "S56": {
            "isTerminatingState": False,
            "ε": "S58"
      },
      "S53": {
            "isTerminatingState": False,
            "ε": [
                  "S51",
                  "S49"
            ]
      },
      "S54": {
            "isTerminatingState": False,
            "ε": "S58"
      },
      "S51": {
            "isTerminatingState": False,
            "7": "S52"
      },
      "S52": {
            "isTerminatingState": False,
            "ε": "S54"
      },
      "S49": {
            "isTerminatingState": False,
            "ε": [
                  "S47",
                  "S45"
            ]
      },
      "S50": {
            "isTerminatingState": False,
            "ε": "S54"
      },
      "S47": {
            "isTerminatingState": False,
            "6": "S48"
      },
      "S48": {
            "isTerminatingState": False,
            "ε": "S50"
      },
      "S45": {
            "isTerminatingState": False,
            "ε": [
                  "S43",
                  "S41"
            ]
      },
      "S46": {
            "isTerminatingState": False,
            "ε": "S50"
      },
      "S43": {
            "isTerminatingState": False,
            "5": "S44"
      },
      "S44": {
            "isTerminatingState": False,
            "ε": "S46"
      },
      "S41": {
            "isTerminatingState": False,
            "ε": [
                  "S39",
                  "S37"
            ]
      },
      "S42": {
            "isTerminatingState": False,
            "ε": "S46"
      },
      "S39": {
            "isTerminatingState": False,
            "4": "S40"
      },
      "S40": {
            "isTerminatingState": False,
            "ε": "S42"
      },
      "S37": {
            "isTerminatingState": False,
            "ε": [
                  "S35",
                  "S33"
            ]
      },
      "S38": {
            "isTerminatingState": False,
            "ε": "S42"
      },
      "S35": {
            "isTerminatingState": False,
            "3": "S36"
      },
      "S36": {
            "isTerminatingState": False,
            "ε": "S38"
      },
      "S33": {
            "isTerminatingState": False,
            "ε": [
                  "S31",
                  "S29"
            ]
      },
      "S34": {
            "isTerminatingState": False,
            "ε": "S38"
      },
      "S31": {
            "isTerminatingState": False,
            "2": "S32"
      },
      "S32": {
            "isTerminatingState": False,
            "ε": "S34"
      },
      "S29": {
            "isTerminatingState": False,
            "ε": [
                  "S27",
                  "S25"
            ]
      },
      "S30": {
            "isTerminatingState": False,
            "ε": "S34"
      },
      "S27": {
            "isTerminatingState": False,
            "1": "S28"
      },
      "S28": {
            "isTerminatingState": False,
            "ε": "S30"
      },
      "S25": {
            "isTerminatingState": False,
            "ε": [
                  "S23",
                  "S21"
            ]
      },
      "S26": {
            "isTerminatingState": False,
            "ε": "S30"
      },
      "S23": {
            "isTerminatingState": False,
            "0": "S24"
      },
      "S24": {
            "isTerminatingState": False,
            "ε": "S26"
      },
      "S21": {
            "isTerminatingState": False,
            "ε": [
                  "S19",
                  "S17"
            ]
      },
      "S22": {
            "isTerminatingState": False,
            "ε": "S26"
      },
      "S19": {
            "isTerminatingState": False,
            "f": "S20"
      },
      "S20": {
            "isTerminatingState": False,
            "ε": "S22"
      },
      "S17": {
            "isTerminatingState": False,
            "ε": [
                  "S15",
                  "S13"
            ]
      },
      "S18": {
            "isTerminatingState": False,
            "ε": "S22"
      },
      "S15": {
            "isTerminatingState": False,
            "e": "S16"
      },
      "S16": {
            "isTerminatingState": False,
            "ε": "S18"
      },
      "S13": {
            "isTerminatingState": False,
            "ε": [
                  "S11",
                  "S9"
            ]
      },
      "S14": {
            "isTerminatingState": False,
            "ε": "S18"
      },
      "S11": {
            "isTerminatingState": False,
            "d": "S12"
      },
      "S12": {
            "isTerminatingState": False,
            "ε": "S14"
      },
      "S9": {
            "isTerminatingState": False,
            "ε": [
                  "S7",
                  "S5"
            ]
      },
      "S10": {
            "isTerminatingState": False,
            "ε": "S14"
      },
      "S7": {
            "isTerminatingState": False,
            "c": "S8"
      },
      "S8": {
            "isTerminatingState": False,
            "ε": "S10"
      },
      "S5": {
            "isTerminatingState": False,
            "ε": [
                  "S3",
                  "S1"
            ]
      },
      "S6": {
            "isTerminatingState": False,
            "ε": "S10"
      },
      "S3": {
            "isTerminatingState": False,
            "ε": "S1"
      },
      "S4": {
            "isTerminatingState": False,
            "ε": "S5"
      },
      "S1": {
            "isTerminatingState": False,
            "a": "S2"
      },
      "S2": {
            "isTerminatingState": False,
            "ε": "S6"
      },
      "S63": {
            "isTerminatingState": False,
            "3": "S64"
      },
      "S64": {
            "isTerminatingState": False,
            "ε": "S65"
      },
      "S65": {
            "isTerminatingState": False,
            "2": "S66"
      },
      "S66": {
            "isTerminatingState": True
      }
}
test_dfa = convert_nfa_to_dfa(test_nfa)
print(test_dfa)
final_dfa = get_final_dfa (test_dfa)
print(final_dfa)


{'startingState': ('S1', 'S11', 'S13', 'S15', 'S17', 'S19', 'S21', 'S23', 'S25', 'S27', 'S29', 'S3', 'S31', 'S33', 'S35', 'S37', 'S39', 'S41', 'S43', 'S45', 'S47', 'S49', 'S5', 'S51', 'S53', 'S55', 'S57', 'S59', 'S61', 'S7', 'S9'), ('S1', 'S11', 'S13', 'S15', 'S17', 'S19', 'S21', 'S23', 'S25', 'S27', 'S29', 'S3', 'S31', 'S33', 'S35', 'S37', 'S39', 'S41', 'S43', 'S45', 'S47', 'S49', 'S5', 'S51', 'S53', 'S55', 'S57', 'S59', 'S61', 'S7', 'S9'): {'isTerminatingState': False, 'a': ('S10', 'S14', 'S18', 'S2', 'S22', 'S26', 'S30', 'S34', 'S38', 'S42', 'S46', 'S50', 'S54', 'S58', 'S6', 'S62', 'S63'), 'd': ('S12', 'S14', 'S18', 'S22', 'S26', 'S30', 'S34', 'S38', 'S42', 'S46', 'S50', 'S54', 'S58', 'S62', 'S63'), 'e': ('S16', 'S18', 'S22', 'S26', 'S30', 'S34', 'S38', 'S42', 'S46', 'S50', 'S54', 'S58', 'S62', 'S63'), 'f': ('S20', 'S22', 'S26', 'S30', 'S34', 'S38', 'S42', 'S46', 'S50', 'S54', 'S58', 'S62', 'S63'), '0': ('S24', 'S26', 'S30', 'S34', 'S38', 'S42', 'S46', 'S50', 'S54', 'S58', 'S62', 'S

# Minimized DFA

In [7]:
def minimize_dfa(dfa):

    unique_inputs = set()

    groups_dic = {}
    groups_dic[1] = set()
    groups_dic[2] = set()

    for key in dfa:
        if key != "startingState":
            if dfa[key]["isTerminatingState"]:
                groups_dic[1].add(key)
            else:
                groups_dic[2].add(key)
        
            for sub_key in dfa[key]:
                if sub_key != "isTerminatingState":
                    unique_inputs.add(sub_key)

    if len(groups_dic[1]) == 0:
        del groups_dic[1]

    if len(groups_dic[2]) == 0:
        del groups_dic[2]

    print(groups_dic,unique_inputs)

    #####################################################

    stop = False

    while not stop:

        stop = True

        groups_names = list(groups_dic.keys())
        
        for group_id in groups_names:
                        
            for input in unique_inputs:

                dic={}
                states_in_group = groups_dic[group_id] 
                print('===========>',states_in_group,'=============>',input,'------------->',groups_dic)

                for state in states_in_group:
                    
                    if input in dfa[state]:
                        next_state = dfa[state][input]

                        for group, state_list in groups_dic.items():
                            if next_state in state_list:
                                if group in dic.keys():
                                    dic[group].add(state)
                                else:
                                    dic[group]={state}
                    else: 
                        if "-" in dic.keys():
                            dic["-"].add(state)
                        else:
                            dic["-"]={state}
                
                print(dic) 

                if len(dic) > 1: # there is division
                    stop = False
                    del groups_dic[group_id]
                    i = max(groups_dic.keys()) + 1
                    for new_group in dic.keys():
                            groups_dic[i]=dic[new_group]
                            i+=1
                    break
            
                        
            
                    
    key_list = list(groups_dic.keys())
    new_dic = {}
    print("groups dic",groups_dic)
    for key, value in groups_dic.items():
        index = key_list.index(key)
        value =  groups_dic[key]  # Get the value associated with the old key
        new_dic["S"+str(index)] = value  # Create a new key-value pair with the new key
        
       

        
    print("groups dic",new_dic)
    inverted_dic = {}
    for key, value in new_dic.items():
        for val in value:
            inverted_dic[val]=key
    
    # inverted_dict = {tuple(value): key for key, value in new_dic.items()}
    minimized_dfa = {
        "startingState":  [key for key, value in new_dic.items() if dfa["startingState"] in value ][0]
    }
    print(minimized_dfa)
    print((dfa["startingState"]))
    print(inverted_dic)
    # Build the minimized dfa in the standard json format specified
    for key, value in new_dic.items():
        # Let the first state in the group be its representative state
        group_first_state = list(value)[0]
        # Equate the transitions of the group with the transitions of its representative state
        minimized_dfa[key] = dfa[group_first_state]
        for inputt in minimized_dfa[key]:
            if inputt == "isTerminatingState":
                continue
            # Change the name of the destination states to the name of their corresponding groups
            minimized_dfa[key][inputt] = inverted_dic[minimized_dfa[key][inputt]]
    print(minimized_dfa)
    return minimized_dfa                        
                    

In [8]:
test_for_min={'startingState': 'S0', 'S0': {'isTerminatingState': False, 'b': 'S4', 'c': 'S1'}, 'S1': {'isTerminatingState': False, 'b': 'S4', 'd': 'S2', 'c': 'S1'}, 'S2': {'isTerminatingState': True, 'c': 'S3'}, 'S3': {'isTerminatingState': False, 'd': 'S2'}, 'S4': {'isTerminatingState': False, 'b': 'S4','c':'S1'}}
try1 = {'startingState': 'S0', 'S0': {'isTerminatingState': False, 'a': 'S1'}, 'S1': {'isTerminatingState': False, 'a': 'S1', 'b': 'S2'}, 'S2': {'isTerminatingState': True}}
minimize_dfa (test_for_min)
# minimize_dfa (try1)
try2 = {'startingState': 'S0', 'S0': {'isTerminatingState': False, 'a': 'S1'}, 'S1': {'isTerminatingState': False, 'b': 'S2'}, 'S2': {'isTerminatingState': False, 'd': 'S5', 'b': 'S4', 'c': 'S3'}, 'S3': {'isTerminatingState': False, 'd': 'S5', 'b': 'S4', 'c': 'S3'}, 'S4': {'isTerminatingState': False, 'd': 'S5', 'b': 'S4', 'c': 'S3'}, 'S5': {'isTerminatingState': True, 'd': 'S5'}}
# minimize_dfa (try2)
# minimize_dfa (final_dfa)

{1: {'S2'}, 2: {'S0', 'S4', 'S3', 'S1'}} {'d', 'b', 'c'}
===========> {'S2'} =============> d -------------> {1: {'S2'}, 2: {'S0', 'S4', 'S3', 'S1'}}
{'-': {'S2'}}
===========> {'S2'} =============> b -------------> {1: {'S2'}, 2: {'S0', 'S4', 'S3', 'S1'}}
{'-': {'S2'}}
===========> {'S2'} =============> c -------------> {1: {'S2'}, 2: {'S0', 'S4', 'S3', 'S1'}}
{2: {'S2'}}
===========> {'S0', 'S4', 'S3', 'S1'} =============> d -------------> {1: {'S2'}, 2: {'S0', 'S4', 'S3', 'S1'}}
{'-': {'S0', 'S4'}, 1: {'S3', 'S1'}}
===========> {'S2'} =============> d -------------> {1: {'S2'}, 2: {'S0', 'S4'}, 3: {'S3', 'S1'}}
{'-': {'S2'}}
===========> {'S2'} =============> b -------------> {1: {'S2'}, 2: {'S0', 'S4'}, 3: {'S3', 'S1'}}
{'-': {'S2'}}
===========> {'S2'} =============> c -------------> {1: {'S2'}, 2: {'S0', 'S4'}, 3: {'S3', 'S1'}}
{3: {'S2'}}
===========> {'S0', 'S4'} =============> d -------------> {1: {'S2'}, 2: {'S0', 'S4'}, 3: {'S3', 'S1'}}
{'-': {'S0', 'S4'}}
===========> {'S0'

In [9]:
test_for_min={'startingState': 'S0', 'S0': {'isTerminatingState': False, 'a': 'S6', 'b': 'S5', 'c': 'S1'}, 'S1': {'isTerminatingState': False, 'd': 'S4', 'e': 'S3', 'f': 'S2'}, 'S2': {'isTerminatingState': True}, 'S3': {'isTerminatingState': True}, 'S4': {'isTerminatingState': True}, 'S5': {'isTerminatingState': False, 'd': 'S4', 'e': 'S3', 'f': 'S2'}, 'S6': {'isTerminatingState': False, 'd': 'S4', 'e': 'S3','f':'S2'}}
minimize_dfa (test_for_min)

{1: {'S2', 'S4', 'S3'}, 2: {'S0', 'S6', 'S5', 'S1'}} {'f', 'b', 'a', 'e', 'c', 'd'}
===========> {'S2', 'S4', 'S3'} =============> f -------------> {1: {'S2', 'S4', 'S3'}, 2: {'S0', 'S6', 'S5', 'S1'}}
{'-': {'S2', 'S4', 'S3'}}
===========> {'S2', 'S4', 'S3'} =============> b -------------> {1: {'S2', 'S4', 'S3'}, 2: {'S0', 'S6', 'S5', 'S1'}}
{'-': {'S2', 'S4', 'S3'}}
===========> {'S2', 'S4', 'S3'} =============> a -------------> {1: {'S2', 'S4', 'S3'}, 2: {'S0', 'S6', 'S5', 'S1'}}
{'-': {'S2', 'S4', 'S3'}}
===========> {'S2', 'S4', 'S3'} =============> e -------------> {1: {'S2', 'S4', 'S3'}, 2: {'S0', 'S6', 'S5', 'S1'}}
{'-': {'S2', 'S4', 'S3'}}
===========> {'S2', 'S4', 'S3'} =============> c -------------> {1: {'S2', 'S4', 'S3'}, 2: {'S0', 'S6', 'S5', 'S1'}}
{'-': {'S2', 'S4', 'S3'}}
===========> {'S2', 'S4', 'S3'} =============> d -------------> {1: {'S2', 'S4', 'S3'}, 2: {'S0', 'S6', 'S5', 'S1'}}
{'-': {'S2', 'S4', 'S3'}}
===========> {'S0', 'S6', 'S5', 'S1'} =============> f ---

{'startingState': 'S1',
 'S0': {'isTerminatingState': True},
 'S1': {'isTerminatingState': False, 'a': 'S2', 'b': 'S2', 'c': 'S2'},
 'S2': {'isTerminatingState': False, 'd': 'S0', 'e': 'S0', 'f': 'S0'}}

In [41]:

from graphviz import Digraph


# def vizualize_NFA(resultNFA):
#   gra = Digraph(graph_attr={'rankdir':'LR'})

#   #construct nodes first
#   for stat in resultNFA.states:
#       if(stat.label == resultNFA.start_state.label):
#         gra.node("", _attributes={'shape' : 'none'})
#         gra.edge("", stat.label)
#       if(stat.label == resultNFA.acceptance_state.label):
#         gra.node(stat.label, _attributes={'peripheries' : '2'})
#       else:
#         gra.node(stat.label)

#   #for each node, construct edges
#   for stat in resultNFA.states:
#       for edg in stat.outgoing_edges:
#           gra.edge(stat.label, edg.destination.label, label=edg.label)
#   gra.format = 'png'
#   gra.render('NFA', view = True)
#   return gra.source
# # Open the JSON file
# with open('NFA.json', 'r') as file:
#     # Load JSON data
#     data = json.load(file)


# vizualize_NFA(nfa1)
nfa1={
      "startingState": "S5",
      "S5": {
            "isTerminatingState": False,
            "ε": [
                  "S3",
                  "S1"
            ]
      },
      "S6": {
            "isTerminatingState": True
      },
      "S3": {
            "isTerminatingState": False,
            "b": "S4"
      },
      "S4": {
            "isTerminatingState": False,
            "ε": "S6"
      },
      "S1": {
            "isTerminatingState": False,
            "a": "S2"
      },
      "S2": {
            "isTerminatingState": False,
            "ε": "S6"
      }
}
render_graph(nfa1, "nfa", graph_attr)
dfa1={'startingState': 'S0', 'S0': {'isTerminatingState': False, 'b': 'S4', 'c': 'S1'}, 'S1': {'isTerminatingState': False, 'b': 'S4', 'd': 'S2', 'c': 'S1'}, 'S2': {'isTerminatingState': True, 'c': 'S3'}, 'S3': {'isTerminatingState': False, 'd': 'S2'}, 'S4': {'isTerminatingState': False, 'b': 'S4','c':'S1'}}
render_graph(dfa1, "dfa", graph_attr)


# test cases 

In [ ]:
#(a*b)(b?a+)
#nfa
postfix = to_postfix("(a*b)(b?a+)")
nfa1 = postfix_to_nfa(postfix)
nfa_dict = convert_nfa_to_dictionary(nfa1)
save_json_file(nfa_dict,"nfa")
render_graph(nfa_dict, "nfa", graph_attr)

#dfa
dfa1 = convert_nfa_to_dfa(nfa1)
save_json_file(dfa1,"dfa")
render_graph(dfa1, "dfa", graph_attr)

#min dfa
minimized_dfa1 = minimize_dfa (dfa1)
save_json_file(minimized_dfa1,"minimized dfa")
render_graph(minimized_dfa1, "minimized dfa", graph_attr)